# Topic Modeling

In [1]:
import pandas as pd
import numpy as np
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [2]:
 %pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [5]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [6]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
list = newsgroups.data
print(len(list))

11314


In [7]:
type(list)

list

In [9]:
df = pd.DataFrame({'text':list})
df.head()

,text
0,I was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh..."
3,\nDo you have Weitek's address/phone number? ...
4,"From article <C5owCB.n3p@world.std.com>, by to..."


In [10]:
df.shape

(11314, 1)

In [11]:
df.text[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

# Cleaning data and making bag of words

In [12]:
corpus = [] # defining corpus
for i in range(0, len(df['text'])): # giving range of values from 0 to 1000
    text = re.sub('[^a-zA-Z]{3,}', ' ', df['text'][i]) # using i for all values instead of 1 column
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    stopword_set = set(stopwords.words('english'))
    text = [ps.stem(word) for word in text if not word in stopword_set]
    text = ' '.join(text)
    corpus.append(text) # finally attach all these cleaned values to corpus directory 

In [13]:
type(corpus)

list

In [14]:
corpus_series = pd.Series(corpus)
corpus_series.head()

0    wonder anyon could enlighten car saw day. door...
1    fair number brave soul upgrad si clock oscil s...
2    well folks, mac plu final gave ghost weekend s...
3    weitek' address/phon number i'd like get infor...
4    articl <c5owcb.n3p@world.std.com tombaker@worl...
dtype: object

# Convert to document-term matrix

In [15]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(corpus)
print(dtm_tf.shape)

(11314, 7443)


In [16]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(dtm_tfidf.shape)

(11314, 7443)


In [17]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=20,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

# Visualizing the models with pyLDAvis

In [18]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.187402  0.081937       1        1  11.634280
13     0.166176  0.030574       2        1  11.121343
17     0.204530 -0.035463       3        1   8.588298
15    -0.002429  0.227005       4        1   7.667652
8      0.199151  0.027471       5        1   7.405098
14     0.048329  0.095262       6        1   6.961852
1      0.077284 -0.044067       7        1   6.129211
9      0.138845 -0.140035       8        1   6.084574
5     -0.145668  0.242875       9        1   5.755774
10    -0.045698  0.045114      10        1   5.153004
4      0.009447 -0.106983      11        1   4.176484
19    -0.041390  0.220393      12        1   3.692146
7      0.113323 -0.041230      13        1   3.654165
16     0.115956 -0.145004      14        1   2.767203
12    -0.194577 -0.194633      15        1   2.155393
0     -0.159970  0.061576      16        1   1.827291
2     -0.182121  0.094001      17        1   1.742720
11    -0.019885 -0.189251      18        1   1.465762
6     -0.224788 -0.075833      19        1   1.090248
3     -0.243918 -0.153706      20        1   0.927502, topic_info=     Category         Freq        Term        Total  loglift  logprob
4029  Default  4721.000000         max  4721.000000  30.0000  30.0000
2116  Default  2402.000000         edu  2402.000000  29.0000  29.0000
2502  Default  2394.000000        file  2394.000000  28.0000  28.0000
7026  Default  6975.000000         use  6975.000000  27.0000  27.0000
2808  Default  2113.000000         god  2113.000000  26.0000  26.0000
2035  Default  1570.000000       drive  1570.000000  25.0000  25.0000
7301  Default  1764.000000      window  1764.000000  24.0000  24.0000
3611  Default  1592.000000         key  1592.000000  23.0000  23.0000
2710  Default  1268.000000        game  1268.000000  22.0000  22.0000
5162  Default  2027.000000     program  2027.000000  21.0000  21.0000
5047  Default  1244.000000       power  1244.000000  20.0000  20.0000
6190  Default  1224.000000       space  1224.000000  19.0000  19.0000
5141  Default  2167.000000     problem  2167.000000  18.0000  18.0000
502   Default  1268.000000       avail  1268.000000  17.0000  17.0000
2909  Default   950.000000         gun   950.000000  16.0000  16.0000
966   Default  1032.000000        card  1032.000000  15.0000  15.0000
7414  Default  2578.000000        year  2578.000000  14.0000  14.0000
3649  Default  3888.000000        know  3888.000000  13.0000  13.0000
1263  Default  1514.000000         com  1514.000000  12.0000  12.0000
6585  Default  1051.000000        team  1051.000000  11.0000  11.0000
3711  Default  1356.000000         law  1356.000000  10.0000  10.0000
377   Default   984.000000    armenian   984.000000   9.0000   9.0000
4819  Default  3660.000000       peopl  3660.000000   8.0000   8.0000
5730  Default  1760.000000        said  1760.000000   7.0000   7.0000
1121  Default  1060.000000   christian  1060.000000   6.0000   6.0000
5701  Default  1757.000000         run  1757.000000   5.0000   5.0000
6690  Default  3360.000000       think  3360.000000   4.0000   4.0000
6182  Default   808.000000       sourc   808.000000   3.0000   3.0000
4954  Default   995.000000        play   995.000000   2.0000   2.0000
1921  Default   795.000000        disk   795.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
660   Topic20   457.214074         bhj   458.160528   4.6784  -2.8176
2783  Topic20   423.999681         giz   424.946135   4.6782  -2.8930
898   Topic20   211.535326         bxn   212.481780   4.6760  -3.5884
5250  Topic20   142.190663         qax   143.137116   4.6738  -3.9856
4496  Topic20   113.235591        nrhj   114.182045   4.6721  -4.2133
700   Topic20    99.915126         biz   100.861580   4.6710  -4.3385
949   Topic20    85.378192       canon    86.324645   4.6694  -4.4957
7388  Topic20    80.062825        wwiz 

In [19]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.070935 -0.062202       1        1  69.393032
9     -0.405520  0.052816       2        1  13.238899
4      0.078354  0.335533       3        1   2.918950
19     0.051090 -0.051795       4        1   1.169144
2      0.033873 -0.027071       5        1   0.910260
0      0.033333 -0.028274       6        1   0.903439
13     0.031573 -0.026357       7        1   0.891609
18     0.029501 -0.025147       8        1   0.886576
16     0.028929 -0.024248       9        1   0.870955
11     0.020684 -0.017063      10        1   0.823281
12     0.020467 -0.016122      11        1   0.819053
1      0.017620 -0.013320      12        1   0.802830
8      0.017160 -0.012826      13        1   0.801173
3      0.016693 -0.012389      14        1   0.797926
17     0.016534 -0.012254      15        1   0.797260
14     0.016537 -0.012257      16        1   0.797149
7      0.016418 -0.012136      17        1   0.796658
10     0.016163 -0.011879      18        1   0.795455
6      0.015762 -0.011504      19        1   0.793176
5      0.015762 -0.011504      20        1   0.793176, topic_info=     Category        Freq       Term       Total  loglift  logprob
2116  Default   93.000000        edu   93.000000  30.0000   30.000
2710  Default   80.000000       game   80.000000  29.0000   29.000
7414  Default  104.000000       year  104.000000  28.0000   28.000
6585  Default   60.000000       team   60.000000  27.0000   27.000
7026  Default  202.000000        use  202.000000  26.0000   26.000
4954  Default   54.000000       play   54.000000  25.0000   25.000
7301  Default  100.000000     window  100.000000  24.0000   24.000
4956  Default   40.000000     player   40.000000  23.0000   23.000
6171  Default   32.000000       soon   32.000000  22.0000   22.000
6664  Default  106.000000      thank  106.000000  21.0000   21.000
2502  Default   81.000000       file   81.000000  20.0000   20.000
2035  Default   89.000000      drive   89.000000  19.0000   19.000
966   Default   74.000000       card   74.000000  18.0000   18.000
7299  Default   43.000000        win   43.000000  17.0000   17.000
5701  Default   81.000000        run   81.000000  16.0000   16.000
4029  Default   23.000000        max   23.000000  15.0000   15.000
300   Default  115.000000      anyon  115.000000  14.0000   14.000
4303  Default   21.000000        msg   21.000000  13.0000   13.000
555   Default   23.000000       bank   23.000000  12.0000   12.000
5141  Default  100.000000    problem  100.000000  11.0000   11.000
3649  Default  164.000000       know  164.000000  10.0000   10.000
2577  Default   21.000000       food   21.000000   9.0000    9.000
5162  Default   73.000000    program   73.000000   8.0000    8.000
5835  Default   27.000000     season   27.000000   7.0000    7.000
7340  Default  110.000000       work  110.000000   6.0000    6.000
3075  Default   26.000000     hockey   26.000000   5.0000    5.000
4963  Default   95.000000      pleas   95.000000   4.0000    4.000
2823  Default   19.000000     gordon   19.000000   3.0000    3.000
4926  Default   18.000000       pitt   18.000000   2.0000    2.000
2424  Default   28.000000        fan   28.000000   1.0000    1.000
...       ...         ...        ...         ...      ...      ...
7432  Topic20    0.060091        yup    6.512668   0.1512   -8.915
2590  Topic20    0.060090       ford   11.318705  -0.4015   -8.915
6689  Topic20    0.060090     things   13.332648  -0.5652   -8.915
498   Topic20    0.060090  automobil    5.691788   0.2859   -8.915
3872  Topic20    0.060090       loop    7.198064   0.0512   -8.915
4284  Topic20    0.060090      motto    5.457522   0.3280   -8.915
3311  Topic20    0.060090     inform   56.289310  -2.0055   -8.915
872   Topic20    0.060090     bumper    4.323580   0.5609   -8.915
6081  Topic20    0.060090       slap    2.966996   0.9374   -8.915
242   Topic20    0.060

# Using different MDS functions
With sklearn installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [20]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.210161  0.266176       1        1  11.634280
13    -0.168710  0.001426       2        1  11.121343
17    -0.160715  0.373155       3        1   8.588298
15     0.055570  0.116932       4        1   7.667652
8     -0.219177  0.179256       5        1   7.405098
14     0.044242  0.327316       6        1   6.961852
1      0.216887 -0.020141       7        1   6.129211
9     -0.373353 -0.002289       8        1   6.084574
5      0.382344  0.249429       9        1   5.755774
10     0.258187  0.168924      10        1   5.153004
4     -0.364013 -0.172832      11        1   4.176484
19    -0.094268 -0.256859      12        1   3.692146
7     -0.397498  0.168176      13        1   3.654165
16     0.103182 -0.193941      14        1   2.767203
12     0.402585 -0.299086      15        1   2.155393
0      0.477526 -0.028812      16        1   1.827291
2      0.215286  0.457621      17        1   1.742720
11     0.179057 -0.399848      18        1   1.465762
6     -0.312677 -0.405583      19        1   1.090248
3     -0.034294 -0.529020      20        1   0.927502, topic_info=     Category         Freq        Term        Total  loglift  logprob
4029  Default  4721.000000         max  4721.000000  30.0000  30.0000
2116  Default  2402.000000         edu  2402.000000  29.0000  29.0000
2502  Default  2394.000000        file  2394.000000  28.0000  28.0000
7026  Default  6975.000000         use  6975.000000  27.0000  27.0000
2808  Default  2113.000000         god  2113.000000  26.0000  26.0000
2035  Default  1570.000000       drive  1570.000000  25.0000  25.0000
7301  Default  1764.000000      window  1764.000000  24.0000  24.0000
3611  Default  1592.000000         key  1592.000000  23.0000  23.0000
2710  Default  1268.000000        game  1268.000000  22.0000  22.0000
5162  Default  2027.000000     program  2027.000000  21.0000  21.0000
5047  Default  1244.000000       power  1244.000000  20.0000  20.0000
6190  Default  1224.000000       space  1224.000000  19.0000  19.0000
5141  Default  2167.000000     problem  2167.000000  18.0000  18.0000
502   Default  1268.000000       avail  1268.000000  17.0000  17.0000
2909  Default   950.000000         gun   950.000000  16.0000  16.0000
966   Default  1032.000000        card  1032.000000  15.0000  15.0000
7414  Default  2578.000000        year  2578.000000  14.0000  14.0000
3649  Default  3888.000000        know  3888.000000  13.0000  13.0000
1263  Default  1514.000000         com  1514.000000  12.0000  12.0000
6585  Default  1051.000000        team  1051.000000  11.0000  11.0000
3711  Default  1356.000000         law  1356.000000  10.0000  10.0000
377   Default   984.000000    armenian   984.000000   9.0000   9.0000
4819  Default  3660.000000       peopl  3660.000000   8.0000   8.0000
5730  Default  1760.000000        said  1760.000000   7.0000   7.0000
1121  Default  1060.000000   christian  1060.000000   6.0000   6.0000
5701  Default  1757.000000         run  1757.000000   5.0000   5.0000
6690  Default  3360.000000       think  3360.000000   4.0000   4.0000
6182  Default   808.000000       sourc   808.000000   3.0000   3.0000
4954  Default   995.000000        play   995.000000   2.0000   2.0000
1921  Default   795.000000        disk   795.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
660   Topic20   457.214074         bhj   458.160528   4.6784  -2.8176
2783  Topic20   423.999681         giz   424.946135   4.6782  -2.8930
898   Topic20   211.535326         bxn   212.481780   4.6760  -3.5884
5250  Topic20   142.190663         qax   143.137116   4.6738  -3.9856
4496  Topic20   113.235591        nrhj   114.182045   4.6721  -4.2133
700   Topic20    99.915126         biz   100.861580   4.6710  -4.3385
949   Topic20    85.378192       canon    86.324645   4.6694  -4.4957
7388  Topic20    80.062825        wwiz 

In [21]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
18     48.623180  -7.138412       1        1  11.634280
13     33.104885 -44.325554       2        1  11.121343
17     94.229591   6.323269       3        1   8.588298
15    -30.583054  12.350696       4        1   7.667652
8     -34.970924 -28.070408       5        1   7.405098
14    -17.976833  49.938110       6        1   6.961852
1     -71.693771  30.046045       7        1   6.129211
9      80.521461 -42.675205       8        1   6.084574
5      12.918296  19.196814       9        1   5.755774
10     -6.858203 -66.707848      10        1   5.153004
4      54.270927  32.223450      11        1   4.176484
19     44.889725  95.395126      12        1   3.692146
7     -55.216961  74.441460      13        1   3.654165
16     22.772209  59.085777      14        1   2.767203
12     39.252392 -86.939346      15        1   2.155393
0      85.581299  61.458717      16        1   1.827291
2      -8.032215  95.959572      17        1   1.742720
11    -54.320038 -67.732796      18        1   1.465762
6       3.320652 -17.717016      19        1   1.090248
3     -78.327629 -16.452925      20        1   0.927502, topic_info=     Category         Freq        Term        Total  loglift  logprob
4029  Default  4721.000000         max  4721.000000  30.0000  30.0000
2116  Default  2402.000000         edu  2402.000000  29.0000  29.0000
2502  Default  2394.000000        file  2394.000000  28.0000  28.0000
7026  Default  6975.000000         use  6975.000000  27.0000  27.0000
2808  Default  2113.000000         god  2113.000000  26.0000  26.0000
2035  Default  1570.000000       drive  1570.000000  25.0000  25.0000
7301  Default  1764.000000      window  1764.000000  24.0000  24.0000
3611  Default  1592.000000         key  1592.000000  23.0000  23.0000
2710  Default  1268.000000        game  1268.000000  22.0000  22.0000
5162  Default  2027.000000     program  2027.000000  21.0000  21.0000
5047  Default  1244.000000       power  1244.000000  20.0000  20.0000
6190  Default  1224.000000       space  1224.000000  19.0000  19.0000
5141  Default  2167.000000     problem  2167.000000  18.0000  18.0000
502   Default  1268.000000       avail  1268.000000  17.0000  17.0000
2909  Default   950.000000         gun   950.000000  16.0000  16.0000
966   Default  1032.000000        card  1032.000000  15.0000  15.0000
7414  Default  2578.000000        year  2578.000000  14.0000  14.0000
3649  Default  3888.000000        know  3888.000000  13.0000  13.0000
1263  Default  1514.000000         com  1514.000000  12.0000  12.0000
6585  Default  1051.000000        team  1051.000000  11.0000  11.0000
3711  Default  1356.000000         law  1356.000000  10.0000  10.0000
377   Default   984.000000    armenian   984.000000   9.0000   9.0000
4819  Default  3660.000000       peopl  3660.000000   8.0000   8.0000
5730  Default  1760.000000        said  1760.000000   7.0000   7.0000
1121  Default  1060.000000   christian  1060.000000   6.0000   6.0000
5701  Default  1757.000000         run  1757.000000   5.0000   5.0000
6690  Default  3360.000000       think  3360.000000   4.0000   4.0000
6182  Default   808.000000       sourc   808.000000   3.0000   3.0000
4954  Default   995.000000        play   995.000000   2.0000   2.0000
1921  Default   795.000000        disk   795.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
660   Topic20   457.214074         bhj   458.160528   4.6784  -2.8176
2783  Topic20   423.999681         giz   424.946135   4.6782  -2.8930
898   Topic20   211.535326         bxn   212.481780   4.6760  -3.5884
5250  Topic20   142.190663         qax   143.137116   4.6738  -3.9856
4496  Topic20   113.235591        nrhj   114.182045   4.6721  -4.2133
700   Topic20    99.915126         biz   100.861580   4.6710  -4.3385
949   Topic20    85.378192       canon    86.324645   4.6694  -4.